# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [125]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "OpenAI"

In [129]:
# Create a messages list using the same format that we used for OpenAI

messages = [
   # {"role": "user", "content": "Describe some of the business applications of Generative AI"}
    {"role": "user", "content": "Describe some benefits of sd-wan compared to mpls network"}
]

In [130]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [115]:
# Let's just make sure the model is loaded

!ollama pull llama3.2:1b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 372 KB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 4.1 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏ 7.0 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏  15 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏  17 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   2% ▕                  ▏  31 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   2% ▕           

In [13]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Here are some benefits of SD-WAN (Software-Defined Wide Area Network) compared to MPLS (Multiprotocol Label Switching) networks:

1. **Cost-effectiveness**: SD-WAN is generally more cost-effective than MPLS, especially for small and medium-sized businesses with a large number of sites. With SD-WAN, you can scale your network without the need for expensive hardware upgrades.

2. **Flexibility and scalability**: SD-WAN is highly scalable and flexible, allowing you to easily add or remove branches as needed. MPLS networks are typically more rigid in their architecture, requiring more significant infrastructure changes to expand.

3. **Ease of deployment**: SD-WAN is often easier to deploy than an MPLS network, which requires physical hardware installation at each site. With SD-WAN, you can deploy the solution remotely and configure it as needed.

4. **Reduced complexity**: SD-WAN simplifies network management by centralizing control and reducing the need for dedicated equipment at each br

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [131]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

ResponseError: model "OpenAI" not found, try pulling it first (status code: 404)

## Alternative approach - using OpenAI python library to connect to Ollama

In [128]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'model "OpenAI" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [17]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  10 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  17 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  19 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  32 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8

In [18]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I'm trying to understand the basic concepts that underpin Large Language Models (LLMs). From what I remember in my studies, LLMs are these advanced AI systems designed to generate text, but they have more than just the core—uh-oh, I need to figure out where exactly. The user asked for definitions of a neural network, attention, and the transformer, all within the context of LLMs.

Hmm, let's start with neural networks since that seems most fundamental. I know that neural networks are inspired by the structure of the human brain. They have layers of artificial neurons that process information through these interconnected nodes. So in the context of an LLM or any NLP system, a neural network would just be one of those models that can learn patterns from text data. But wait, how does this relate to machine learning and deep learning? Maybe it's more about structured models rather than the traditional feed-forward approach used sometimes.

Now, attention—this is something 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [132]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connect to model
MODEL = "deepseek-r1:1.5b"
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

# See how this function creates exactly the format above
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [133]:
display_summary("https://cnn.com")

<think>
Alright, I need to analyze the given website content and provide a short summary, especially focusing on news and announcements.

Looking through the site's content, I see sections like Breaking News, Latest News, and Videos. I'll start by identifying key information in each relevant section.

Breaking News: The main event here is the U.S. government calling for an emergency to prevent U.S.-Iranian military ties from straining relations with both Iran and U.S. voters. This seems like a crucial news item highlighting bilateral tensions and foreign policy concerns.

Latest News: There isn't much recent activity besides the Breaking News piece, so I'll focus on that.

Video section: The website mentions an ad preventing or slowing page loading (5:07). This could be related to user-friendly design or technical issues. Also, a video about two-time defending champions Barcelona winning their first Women's Champions League title in 18 years seems exciting and matches CNN's theme of tech and sports.

Sports section: The article discusses a case where twin 19-year-olds were found dead on a remote Georgia mountain-top. It points out that police now say they've solved the mystery, mentioning twin disappearances. This shows ongoing investigations into similar tragic incidents.

Looking for other relevant sections like Features or Expert Insights, nothing stands out as critical unless specific details are missing from the articles mentioned.

In terms of structure, CNN is known for covering sports extensively and sometimes tech topics with timely content. Given the absence of detailed stats on the football case or technical specifics in the video ad, I might highlight it less but include the notable feature about the Champions League victory.

Putting it all together, the main news points to a border-line policy shift in U.S.-Iran relations, an international sports event involving twin disappearances, and a newly discovered sport-related tragedy. These aspects are significant enough to warrant coverage but require careful selection based on current events.
</think>

CNN recently reported **the urgent call from the U.S. government to address U.S.-Iranian military ties**, emphasizing efforts to prevent foreign relations strain with both Iran and U.S. voters. The site also highlighted a **borderline policy shift in U.S.-Iranian relations** related to military conflicts, likely impacting U.S. political dynamics.

Inside the video section, there's notable mention of the **ad preventing or slowing page loading**, suggesting potential user interface issues. Additionally, a sports feature on **two-time defending champions Barcelona winning their first Women’s Champions League title in 18 years** stands out as an exciting achievement in sports history.

The site also revealed details about a **tragedical case involving twin disappearances**, with police now attributing the mystery to their investigations. While comprehensive, some articles focus on less current events like the U.S.-Iran news and sport-specific results (like Barcelona's victory), providing a balanced overview of CNN's content coverage in sports and technology.